In [26]:
# Authenticate Earth Engine (Uncomment on first run)
import ee
ee.Authenticate()

True

In [27]:
import geemap

ee.Initialize(project='ee-gis1-ukraine')

# Create the map
Map = geemap.Map(center=[40, -100], zoom=4)

Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

In [16]:
# Get Ukraine boundary
ukraine = ee.FeatureCollection('FAO/GAUL/2015/level0') \
    .filter(ee.Filter.eq('ADM0_NAME', 'Ukraine'))

# Get Sentinel-2 image collection for June 2019
image = ee.ImageCollection('COPERNICUS/S2') \
    .filterDate('2019-06-01', '2019-06-30') \
    .filterBounds(ukraine) \
    .median() \
    .clip(ukraine)

# Add NDVI layer
ndvi = image.normalizedDifference(['B8', 'B4']).rename('ndvi')
image = image.addBands(ndvi)

# Visualization parameters for NDVI
ndviParams = {
    'min': -1,
    'max': 1,
    'palette': ['blue', 'white', 'green']
}

# Add NDVI layer to map
Map.addLayer(image.select('ndvi'), ndviParams, 'NDVI')

# Set center and zoom level for the map
Map.centerObject(ukraine, 6)

# Display the map
Map

Map(bottom=498.0, center=[49.033845999377135, 31.488846852735307], controls=(WidgetControl(options=['position'…

In [48]:
# Indices: NDVI, MSAVI, EVI

def get_monthly_NDVI(year, month):
    # Get Ukraine boundary
    ukraine = ee.FeatureCollection('FAO/GAUL/2015/level0') \
        .filter(ee.Filter.eq('ADM0_NAME', 'Ukraine'))

    # Get Sentinel-2 image collection
    image = ee.ImageCollection('COPERNICUS/S2') \
        .filterDate(f'{year}-{month}-01', f'{year}-{month}-30') \
        .filterBounds(ukraine) \
        .median() \
        .clip(ukraine)

    # Add NDVI layer
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('ndvi')
    image = image.addBands(ndvi)

    # Visualization parameters for NDVI
    ndviParams = {
        'min': -1,
        'max': 1,
        'palette': ['blue', 'white', 'green']
    }
    
    return image.select('ndvi').visualize(**ndviParams), ndviParams

def get_monthly_EVI(year, month, mask_clouds=False):
    # Get Ukraine boundary
    ukraine = ee.FeatureCollection('FAO/GAUL/2015/level0') \
        .filter(ee.Filter.eq('ADM0_NAME', 'Ukraine'))

    # Get Sentinel-2 image collection (Level 1C)
    image = ee.ImageCollection('COPERNICUS/S2') \
        .filterDate(f'{year}-{month}-01', f'{year}-{month}-30') \
        .filterBounds(ukraine) \
        .median() \
        .clip(ukraine)

    # Mask clouds if required using QA60
    if mask_clouds:
        # Get the QA60 band for cloud masking and convert to integer type
        qa60 = image.select('QA60').toInt()

        # Create a cloud mask (bit 10 of QA60 indicates cloud cover)
        cloud_mask = qa60.bitwiseAnd(1 << 10).eq(0)

        # Apply the cloud mask to the image
        image = image.updateMask(cloud_mask)

    # Add EVI layer
    evi = image.expression(
        '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
            'NIR': image.select('B8'),
            'RED': image.select('B4'),
            'BLUE': image.select('B2')
        }).rename('evi')
    image = image.addBands(evi)

    # Visualization parameters for EVI, palette should be F5ECD5, 578E7E, 809D3C
    eviParams = {
        'min': -10,
        'max': 10,
        'palette': ['F5ECD5', '578E7E', '809D3C']
    }
    return image.select('evi').visualize(**eviParams), eviParams



image, ndviParams = get_monthly_EVI(2019, 6, mask_clouds=True)
Map.addLayer(image, {}, 'EVI')
Map

Map(bottom=2917217.0, center=[47.97369142346794, 37.46273517608643], controls=(WidgetControl(options=['positio…

In [44]:
# compare EVI from 2016 and 2024
image_2016, eviParams_2016 = get_monthly_EVI(2016, 6)
image_2024, eviParams_2024 = get_monthly_EVI(2016, 7)

# calculate a difference between 2024 and 2016
evi_diff = image_2024.subtract(image_2016)

# Visualization parameters for EVI difference
eviDiffParams = {
    'min': -200,
    'max': 0
}

Map.addLayer(evi_diff, eviDiffParams, 'EVI Difference')
Map


Map(bottom=11617.0, center=[48.53843177405046, 32.97546386718751], controls=(WidgetControl(options=['position'…